In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [28]:
newdict={}
newdict["intents"]=[
    {"tag":"greeting",
    "patterns":["hi","hello","hi there","hey","are you there?","yo","whats up"],
    "responses":["Hello","Hi","How can i help?","Hey"],
    "context":[""]
    },
    {"tag":"goodbye",
    "patterns":["bye","see ya later","goodbye","ok bye"],
    "responses":["Bye","Goodbye","Come back again soon","Nice to meet you.Bye!"],
    "context":[""]
    },
    {"tag":"noanswer",
    "patterns":[],
    "responses":["Please give me more info","Sorry i cant understand you"],
    "context":[""]
    },
    {"tag":"job",
    "patterns":["what you do?","how can u help me?","what are you gonna do"],
    "responses":["I can guide u through Artificial Intelligence(AI),Data Science and Natural Language Processing"],
    "context":[""]
    },
    {"tag":"thanks",
    "patterns":["thanks","thank you","thanks for the help","thats helpful"],
    "responses":["My Pleasure","You are Welcome","Happy to help :)"],
    "context":[""]
    }
]

In [29]:
print(newdict)

{'intents': [{'tag': 'greeting', 'patterns': ['hi', 'hello', 'hi there', 'hey', 'are you there?', 'yo', 'whats up'], 'responses': ['Hello', 'Hi', 'How can i help?', 'Hey'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['bye', 'see ya later', 'goodbye', 'ok bye'], 'responses': ['Bye', 'Goodbye', 'Come back again soon', 'Nice to meet you.Bye!'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ['Please give me more info', 'Sorry i cant understand you'], 'context': ['']}, {'tag': 'job', 'patterns': ['what you do?', 'how can u help me?', 'what are you gonna do'], 'responses': ['I can guide u through Artificial Intelligence(AI),Data Science and Natural Language Processing'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['thanks', 'thank you', 'thanks for the help', 'thats helpful'], 'responses': ['My Pleasure', 'You are Welcome', 'Happy to help :)'], 'context': ['']}]}


In [30]:
import json
intents=json.dumps(newdict)
with open("intents.json","w") as f:
    f.write(intents)

In [31]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']
data_file=open('intents.json').read()
intents=json.loads(data_file)

In [32]:
for intent in intents['intents']:
    print(intent['tag'])

greeting
goodbye
noanswer
job
thanks


In [33]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        #add tags in corpus
        documents.append((w,intent['tag']))
        #add classes to our tags
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [34]:
#lemmatize each word and remove duplicates
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))
#sort classes
classes=sorted(list(set(classes)))
#documents = combination between patterns and intents
print(len(documents), "documents")
#classes =intents
print (len(classes), "classes", classes)
#words =all words,vocabulary
print(len(words), "unique lemmatized words", words)

18 documents
4 classes ['goodbye', 'greeting', 'job', 'thanks']
30 unique lemmatized words ['are', 'bye', 'can', 'do', 'for', 'gon', 'goodbye', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'later', 'me', 'na', 'ok', 'see', 'thank', 'thanks', 'thats', 'the', 'there', 'u', 'up', 'what', 'whats', 'ya', 'yo', 'you']


In [35]:
#store it in pickle
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [38]:
#create training data
training=[]
output_empty=[0]*len(classes)
for doc in documents:
    bag=[]
    pattern_words=doc[0]
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])
# shuffle our features and turn into np.array])
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x=list(training[:,0])
train_y=list(training[:,1])

In [43]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist=model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)
print("model created")

Epoch 1/100
18/18 [==============================] - 0s 14ms/step - loss: 1.3894 - accuracy: 0.2222
Epoch 2/100
18/18 [==============================] - 0s 1ms/step - loss: 1.3263 - accuracy: 0.3889
Epoch 3/100
18/18 [==============================] - 0s 754us/step - loss: 1.2709 - accuracy: 0.5000
Epoch 4/100
18/18 [==============================] - 0s 471us/step - loss: 1.2042 - accuracy: 0.6667
Epoch 5/100
18/18 [==============================] - 0s 412us/step - loss: 1.2670 - accuracy: 0.4444
Epoch 6/100
18/18 [==============================] - 0s 509us/step - loss: 1.1501 - accuracy: 0.5556
Epoch 7/100
18/18 [==============================] - 0s 604us/step - loss: 1.2492 - accuracy: 0.6111
Epoch 8/100
18/18 [==============================] - 0s 851us/step - loss: 1.0966 - accuracy: 0.6667
Epoch 9/100
18/18 [==============================] - 0s 356us/step - loss: 1.0270 - accuracy: 0.6667
Epoch 10/100
18/18 [==============================] - 0s 532us/step - loss: 1.0291 - accuracy:

18/18 [==============================] - 0s 499us/step - loss: 0.1099 - accuracy: 1.0000
Epoch 82/100
18/18 [==============================] - 0s 450us/step - loss: 0.0202 - accuracy: 1.0000
Epoch 83/100
18/18 [==============================] - 0s 444us/step - loss: 0.0400 - accuracy: 1.0000
Epoch 84/100
18/18 [==============================] - 0s 575us/step - loss: 0.1038 - accuracy: 0.9444
Epoch 85/100
18/18 [==============================] - 0s 485us/step - loss: 0.0583 - accuracy: 0.9444
Epoch 86/100
18/18 [==============================] - 0s 346us/step - loss: 0.0211 - accuracy: 1.0000
Epoch 87/100
18/18 [==============================] - 0s 805us/step - loss: 0.0565 - accuracy: 1.0000
Epoch 88/100
18/18 [==============================] - 0s 934us/step - loss: 0.0425 - accuracy: 1.0000
Epoch 89/100
18/18 [==============================] - 0s 866us/step - loss: 0.0317 - accuracy: 1.0000
Epoch 90/100
18/18 [==============================] - 0s 672us/step - loss: 0.0221 - accuracy: 